# 8-reinas Genético

In [178]:
import numpy as np
import random

In [179]:
class Individual:
    '''
    Individuo del algoritmo genético para el problema de las 8 reinas.
    '''
    
    def __init__(self, chromosome=None):
        self.size = 8 # Tamaño del tablero (8x8)
        if chromosome is None:
            self.chromosome = self._generate_chromosome()
        else:
            self.chromosome = chromosome

        
    def _generate_chromosome(self):
        return random.sample(range(self.size), self.size)
    
    def fitness(self):
        fitness = 0
        for i in range(self.size):
            for j in range(i+1, self.size):
                if self.chromosome[i] == self.chromosome[j]:
                    fitness += 1
                offset = j - i
                if self.chromosome[i] == self.chromosome[j] - offset or self.chromosome[i] == self.chromosome[j] + offset:
                    fitness += 1
        return fitness


    def print_chromosome(self):
        print(self.chromosome)

    def print_board(self):
        board = np.zeros((self.size, self.size), dtype=int)
        for i in range(self.size):
            board[i][self.chromosome[i]] = 1

        for i in range(self.size):
            for j in range(self.size):
                print(board[i][j], end=' ')
            print()
         

In [192]:
class Population:

    def __init__(self, popilation_size, mutation_rate, crossover_rate, num_crosses):
        self.popilation_size = popilation_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.num_crosses = num_crosses
        self.population = [Individual() for _ in range(self.popilation_size)]
        self.fitness_scores = []
        self.evaluate_fitness()

    def evaluate_fitness(self):
        self.fitness_scores = [ind.fitness() for ind in self.population]
        
    def selection(self):
        total_fitness = sum(self.fitness_scores)
        if total_fitness == 0:
            return self.individuals
        selection_probs = [f / total_fitness for f in self.fitness_scores]
        return random.choices(self.population, weights=selection_probs, k=2)

    def crossover(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            crossover_point = random.randint(1, self.population[0].size - 2)  # Punto de cruce entre 1 y 6
            child_chromosome = parent1.chromosome[:crossover_point] + parent2.chromosome[crossover_point:]
            return Individual(child_chromosome)
        else:
            # En caso de no cruzar, elegimos uno de los padres al azar para pasar a la siguiente generación
            return random.choice([parent1, parent2])
        
    def mutate(self, individual):
        if random.random() < self.mutation_rate:
            mutation_point = random.randint(0, self.population[0].size - 1)  # Índice de mutación entre 0 y 7
            new_value = random.randint(0, self.population[0].size - 1)  # Nuevo valor también entre 0 y 7
            # Evitar auto-mutación asignando el mismo valor
            while new_value == individual.chromosome[mutation_point]:
                new_value = random.randint(0, self.population[0].size - 1)
            individual.chromosome[mutation_point] = new_value
        return individual

    def generate_new_generation(self):

        new_population = []

        for i in range(self.num_crosses):
            selected_individuals = self.selection()

            parent1, parent2 = selected_individuals[0], selected_individuals[1]
            child1 = self.crossover(parent1, parent2)
            child2 = self.crossover(parent1, parent2)

            # añadir los hijos a la población y eliminar los peores Individuals para mantener la poblacion 
            new_population.append(self.mutate(child1))
            new_population.append(self.mutate(child2))

        # add new_population to population
        self.population.extend(new_population)

        self.population.sort(key=lambda x: x.fitness())
        self.population = self.population[:self.popilation_size]
        self.evaluate_fitness()

In [193]:
def run_genetic_algorithm(population_size, mutation_rate, crossover_rate, max_generations, num_crosses):
    # Crear la población inicial
    population = Population(population_size, mutation_rate, crossover_rate, num_crosses)

    generation = 0
    while generation < max_generations:
        # Imprime el estado actual
        best_fit = min(population.fitness_scores)
        # print(f"Generación {generation}: Mejor aptitud = {best_fit}")

        # Verifica si hay una solución óptima
        if best_fit == 0:
            best_individual_index = population.fitness_scores.index(best_fit)
            best_individual = population.population[best_individual_index]
            print("Se encontró una solución óptima:")
            best_individual.print_board()
            print("Generación:", generation)
            break

        # Genera una nueva generación
        population.generate_new_generation()
        
        generation += 1

    if generation == max_generations:
        print("Se alcanzó el número máximo de generaciones sin encontrar una solución óptima.")
        # print the best individual (less number of attacks
        # )
        best_individual_index = population.fitness_scores.index(best_fit)
        best_individual = population.population[best_individual_index]
        print("El mejor Individual es:")
        best_individual.print_board()
        print("Fitness:", best_individual.fitness())

In [198]:
# Configuración de parámetros
population_size = 20 # Tamaño de la población
mutation_rate = 0.1  # 10% de probabilidad de mutación
crossover_rate = 0.9  # 90% de probabilidad de cruce
max_generations = 5000 # Número máximo de generaciones
num_crosses = 5 # Número de cruces por generación

# Ejecución del algoritmo genético
run_genetic_algorithm(population_size, mutation_rate, crossover_rate, max_generations, num_crosses)

Se encontró una solución óptima:
0 1 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 
0 0 0 0 0 1 0 0 
0 0 0 0 0 0 0 1 
0 0 1 0 0 0 0 0 
1 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 0 
0 0 0 0 1 0 0 0 
Generación: 73
